In [139]:
!pip install rouge-score transformers

In [140]:
%cd '/content/drive/MyDrive/Colab Notebooks/NLP_HW2'

/content/drive/MyDrive/Colab Notebooks/NLP_HW2


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
#          오늘    점심   엄청   맛있다   또   먹자
enc_ids = torch.tensor([[21882, 482, 5221, 11482, 899, 7221, 1, 1, 1, 1]])
enc_mask = torch.tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])
#             점심  맛있다
dec_ids = torch.tensor([[0, 482, 11482, 1, 1]])
dec_mask = torch.tensor([[1, 1, 1, 0, 0]])
label_ids = torch.tensor([[482, 11482, 2, -100, -100]])

#batch size 1, encoder length 10, decoder length 5

In [ ]:
enc_ids.shape

torch.Size([1, 10])

In [ ]:
embed_tokens = nn.Embedding(50265, 10, 1)
#embed_tokens.weight
encoder_input = embed_tokens(enc_ids)
encoder_input.shape

torch.Size([1, 10, 10])

# encoder

In [ ]:
def _expand_mask(mask, tgt_len = None):
    """
        Inputs
            mask.shape = (B, S_L)
        Outputs
            output.shape = (B, 1, T_L, S_L)
    """
    batch_size, src_len = mask.size()
    tgt_len = tgt_len if tgt_len is not None else src_len

    expanded_mask = mask[:, None, None, :].expand(batch_size, 1, tgt_len, src_len).to(torch.float)

    inverted_mask = 1.0 - expanded_mask

    return inverted_mask.masked_fill(inverted_mask.bool(), torch.finfo(torch.float).min)

In [ ]:
expand_mask = _expand_mask(enc_mask)
print(expand_mask)
print(expand_mask.shape)

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
      

# multihead attention

In [ ]:
#Encoder layer forward(enc_ids, expand_mask)
padding_mask = expand_mask

#attention = self.self_attn(hidden_states, hidden_states, padding_mask)
query, key, value, mask = encoder_input, encoder_input, encoder_input, padding_mask

query


tensor([[[ 0.0725,  0.6345,  0.8128, -0.4669, -0.2900, -1.3800, -0.3416,
           0.6826,  0.1324,  0.3643],
         [-2.3164,  0.2476,  0.6994, -0.7209,  1.2377,  0.0372,  1.5310,
          -0.2016,  1.2990, -0.9954],
         [ 0.3097,  0.5691,  0.8059,  1.5101, -0.8428, -1.1825,  0.5196,
          -0.7080, -1.7979, -0.7161],
         [-0.0895, -1.2875,  0.5487,  0.9424, -0.3219, -1.5662,  0.3610,
           0.3574,  0.7317, -0.0905],
         [-0.7972,  1.2742,  1.1036, -0.8741, -1.0072,  1.3630, -0.6574,
          -2.4297, -0.2394,  1.5738],
         [-1.4687, -0.3526, -1.4621, -0.1650,  0.2930,  0.9102, -0.1139,
           0.5029,  0.1136,  0.5212],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
 

In [ ]:
batch_size = 1
num_heads = 5
d_model = 10 #feature
d_head = 2
#점심(x1x2), 먹는다(x1x2)
#점심(x3x4), 먹는다(x3x4).. 이렇게 나눠짐
query_split = query.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)
key_split = key.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)
value_split = value.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)

print(query_split)
print(query_split.shape)

tensor([[[[ 0.0725,  0.6345],
          [-2.3164,  0.2476],
          [ 0.3097,  0.5691],
          [-0.0895, -1.2875],
          [-0.7972,  1.2742],
          [-1.4687, -0.3526],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[ 0.8128, -0.4669],
          [ 0.6994, -0.7209],
          [ 0.8059,  1.5101],
          [ 0.5487,  0.9424],
          [ 1.1036, -0.8741],
          [-1.4621, -0.1650],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[-0.2900, -1.3800],
          [ 1.2377,  0.0372],
          [-0.8428, -1.1825],
          [-0.3219, -1.5662],
          [-1.0072,  1.3630],
          [ 0.2930,  0.9102],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[-0.3416,  0.6826],
          [ 1.5310, -0.2016],
          [ 0.5196, -0.7080],
    

In [ ]:
padding_mask

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00, -3.4028e+38, -3.4028e+38, -3.4028e+38, -3.4028e+38],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
      

In [ ]:
d_K = torch.tensor(key.size()[-1])
scores = torch.matmul(query_split, key_split.permute(0, 1, 3, 2)) / torch.sqrt(d_K)

scores
#scores는 
#[
#[head1[head2 [head3 [head4
#점맛또먹 점맛또먹 점맛또먹 점맛또먹
#[xxxx],[xxxx],[xxxx],[xxxx]      점
#[xxxx],[xxxx],[xxxx],[xxxx]      맛
#[xxxx],[xxxx],[xxxx],[xxxx]      또
#[xxxx],[xxxx],[xxxx],[xxxx]      먹  유사도.
#      ]      ]      ]      ]]]

tensor([[[[ 1.2897e-01, -3.4307e-03,  1.2129e-01, -2.6038e-01,  2.3737e-01,
           -1.0442e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-3.4307e-03,  1.7162e+00, -1.8232e-01, -3.5242e-02,  6.8376e-01,
            1.0482e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.2129e-01, -1.8232e-01,  1.3276e-01, -2.4048e-01,  1.5123e-01,
           -2.0731e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.6038e-01, -3.5242e-02, -2.4048e-01,  5.2673e-01, -4.9620e-01,
            1.8513e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 2.3737e-01,  6.8376e-01,  1.5123e-01, -4.9620e-01,  7.1440e-01,
            2.2821e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-1.0442e-01,  1.0482e+00, -2.0731e-01,  1.8513e-01,  2.2821e-01,
            7.2145e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
      

In [ ]:
if padding_mask is not None:
    scores = scores.masked_fill(mask!=0,-1e9)
attention = torch.softmax(scores, dim=-1)

print(attention)
print(attention.shape)
#attention: [n_batch, n_head, n_Q, n_K]



tensor([[[[0.1834, 0.1607, 0.1820, 0.1243, 0.2044, 0.1452, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0755, 0.4217, 0.0632, 0.0732, 0.1502, 0.2162, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1924, 0.1420, 0.1947, 0.1340, 0.1983, 0.1385, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1279, 0.1601, 0.1304, 0.2809, 0.1010, 0.1996, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1524, 0.2381, 0.1398, 0.0732, 0.2455, 0.1510, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.0992, 0.3140, 0.0895, 0.1325, 0.1383, 0.2265, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000

In [ ]:
dropout = nn.Dropout(0.1)
out_proj = nn.Linear(d_model, d_model)

In [ ]:
query.shape

torch.Size([1, 10, 10])

In [ ]:
x_raw = torch.matmul(attention,value_split)
x_rsh1 = x_raw.permute(0,2,1,3).contiguous()

# 4. 헤드 concatenate하기
# (batch_size, query의 문장 길이, d_model)
x_concat = x_rsh1.view(batch_size,-1,d_model)


# 5. WO에 해당하는 밀집층 지나기
attn_output = out_proj(x_concat)

print(attn_output)
print(attn_output.shape)


tensor([[[ 0.1671,  0.2901, -0.4598, -0.4990, -0.3311,  0.1152,  0.0412,
          -0.2116, -0.0966,  0.5168],
         [-0.1487,  0.2101, -0.2655, -0.6353, -0.3209, -0.0547,  0.3438,
           0.0212,  0.0464,  0.7568],
         [ 0.3181,  0.6471, -0.2273, -0.4653, -0.6953, -0.0240, -0.2850,
           0.3174,  0.2288,  0.6083],
         [-0.0266,  0.3176, -0.1991, -0.5473, -0.3864,  0.3357,  0.0489,
          -0.1126, -0.0554,  0.4870],
         [-0.2337,  0.8795, -0.3797, -0.0260, -0.4860, -0.4501,  0.2165,
           0.3846,  0.4030,  0.8394],
         [ 0.0014,  0.3439, -0.2506, -0.4849, -0.3767, -0.2105,  0.1598,
           0.0182, -0.0164,  0.4932],
         [ 0.0677,  0.3783, -0.3094, -0.4257, -0.3768, -0.0086,  0.0378,
           0.0214,  0.0708,  0.5178],
         [ 0.0677,  0.3783, -0.3094, -0.4257, -0.3768, -0.0086,  0.0378,
           0.0214,  0.0708,  0.5178],
         [ 0.0677,  0.3783, -0.3094, -0.4257, -0.3768, -0.0086,  0.0378,
           0.0214,  0.0708,  0.5178],
 

In [ ]:
enc_hidden_states = attn_output

# decoder

In [ ]:
decoder_input = embed_tokens(dec_ids)
decoder_input.shape

#decoder_input
#[ 점심 , 맛있다, 2, 0, 0]
#[[xxxxx],[xxxxx],[xxxxx]..]

torch.Size([1, 5, 10])

In [ ]:
dec_expand_mask = _expand_mask(enc_mask, dec_ids.shape[-1])
dec_expand_mask.shape

torch.Size([1, 1, 5, 10])

In [ ]:
def _make_causal_mask(dec_ids):
    """
        Inputs
            dec_ids.shape = (B, D_L)
        Outputs
            output.shape = (B, 1, D_L, D_L)
    """
    batch_size, tgt_len = dec_ids.size()
    device = dec_ids.device

    mask = torch.full((tgt_len, tgt_len), float("-inf"))
    mask_cond = torch.arange(mask.size(-1))
    mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 0)
    mask = mask.to(torch.float).to(device)

    return mask[None, None, :, :].expand(batch_size, 1, tgt_len, tgt_len)

In [ ]:
dec_causal = _make_causal_mask(dec_ids)
print(dec_causal)
print(dec_causal.shape)

#룩어헤드 마스크
#  
#[[[
#  점심 맛있다 0 0 0 
#   [o x x x x],
#   [o o x x x],
#   [o o o x x],
#   [o o o o x],
#   [o o o o o]]]]

tensor([[[[0., -inf, -inf, -inf, -inf],
          [0., 0., -inf, -inf, -inf],
          [0., 0., 0., -inf, -inf],
          [0., 0., 0., 0., -inf],
          [0., 0., 0., 0., 0.]]]])
torch.Size([1, 1, 5, 5])


In [ ]:
#hidden_states = decoder_layer(hidden_states, dec_self_mask, enc_hidden_states, enc_dec_mask)
#                              decoder_input    dec_causal 

In [ ]:
look_ahead_mask = dec_causal

query, key, value, mask = decoder_input, decoder_input, decoder_input, look_ahead_mask

In [ ]:
query.shape

torch.Size([1, 5, 10])

In [ ]:
batch_size = 1
num_heads = 5
d_model = 10 #feature
d_head = 2
query_split = query.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)
key_split = key.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)
value_split = value.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)

print(query_split)
print(query_split.shape)

tensor([[[[ 0.2355, -0.0143],
          [-2.3164,  0.2476],
          [-0.0895, -1.2875],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[ 0.0716,  0.0137],
          [ 0.6994, -0.7209],
          [ 0.5487,  0.9424],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[ 0.2355, -1.0561],
          [ 1.2377,  0.0372],
          [-0.3219, -1.5662],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[ 0.6391,  1.8472],
          [ 1.5310, -0.2016],
          [ 0.3610,  0.3574],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]],

         [[ 0.8236,  0.9426],
          [ 1.2990, -0.9954],
          [ 0.7317, -0.0905],
          [ 0.0000,  0.0000],
          [ 0.0000,  0.0000]]]], grad_fn=<PermuteBackward0>)
torch.Size([1, 5, 5, 2])


In [ ]:
d_K = torch.tensor(key.size()[-1])
scores = torch.matmul(query_split, key_split.permute(0, 1, 3, 2)) / torch.sqrt(d_K)
scores

#scores는 
#[
#[head1[head2 [head3 [head4
#점맛00 점맛00 점맛00 점맛00 
#[xxxx],[xxxx],[xxxx],[xxxx]      점
#[xxxx],[xxxx],[xxxx],[xxxx]      맛
#[xxxx],[xxxx],[xxxx],[xxxx]      0
#[xxxx],[xxxx],[xxxx],[xxxx]      0  유사도.
#      ]      ]      ]      ]]]

tensor([[[[ 1.7600e-02, -1.7361e-01, -8.3319e-04,  0.0000e+00,  0.0000e+00],
          [-1.7361e-01,  1.7162e+00, -3.5242e-02,  0.0000e+00,  0.0000e+00],
          [-8.3319e-04, -3.5242e-02,  5.2673e-01,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 1.6821e-03,  1.2729e-02,  1.6505e-02,  0.0000e+00,  0.0000e+00],
          [ 1.2729e-02,  3.1902e-01, -9.3475e-02,  0.0000e+00,  0.0000e+00],
          [ 1.6505e-02, -9.3475e-02,  3.7605e-01,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 3.7023e-01,  7.9745e-02,  4.9908e-01,  0.0000e+00,  0.0000e+00],
          [ 7.9745e-02,  4.8485e-01, -1.4439e-01,  0.0000e+00,  0.0000e+00],
          [ 4.9908e-01, -1.4439e-01,  8.0845e-01,  0.0000e+00,  0.0000e+

In [ ]:
if mask is not None:
    scores = scores.masked_fill(mask!=0,-1e9)

scores

#scores는 
#[
#[head1[head2 [head3 [head4
#점맛00 점맛00 점맛00 점맛00 
#[1xxx],[xxxx],[xxxx],[xxxx]      점
#[12xx],[xxxx],[xxxx],[xxxx]      맛
#[123x],[xxxx],[xxxx],[xxxx]      0
#[1234],[xxxx],[xxxx],[xxxx]      0  유사도.
#      ]      ]      ]      ]]]

tensor([[[[ 1.7600e-02, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [-1.7361e-01,  1.7162e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [-8.3319e-04, -3.5242e-02,  5.2673e-01, -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 1.6821e-03, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 1.2729e-02,  3.1902e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 1.6505e-02, -9.3475e-02,  3.7605e-01, -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 3.7023e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 7.9745e-02,  4.8485e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [ 4.9908e-01, -1.4439e-01,  8.0845e-01, -1.0000e+09, -1.0000e+

In [ ]:
attention = torch.softmax(scores, dim=-1)

print(attention)
print(attention.shape)
#attention: [n_batch, n_head, n_Q, n_K]

#scores는 
#[batch
#[head1[head2 [head3 [head4
#점맛00 점맛00 점맛00 점맛00 
#[1xxx],[xxxx],[xxxx],[xxxx]      점 
#[12xx],[xxxx],[xxxx],[xxxx]      맛
#[123x],[xxxx],[xxxx],[xxxx]      0
#[1234],[xxxx],[xxxx],[xxxx]      0  유사도.
#      ]      ]      ]      ]]]
# ---- 이 방향으로 정규화

#점심, 점심 유사도
#점심, 맛있다 유사도, 맛있다, 맛있다 유사도  

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.1313, 0.8687, 0.0000, 0.0000, 0.0000],
          [0.2732, 0.2639, 0.4629, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
          [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4240, 0.5760, 0.0000, 0.0000, 0.0000],
          [0.3004, 0.2691, 0.4304, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
          [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4001, 0.5999, 0.0000, 0.0000, 0.0000],
          [0.3463, 0.1819, 0.4718, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
          [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]],

         [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.3630, 0.6370, 0.0000, 0.0000, 0.0000],
          [0.3708, 0.3257, 0.3035, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500, 0.0000],
      

In [ ]:
x_raw = torch.matmul(attention,value_split)
x_rsh1 = x_raw.permute(0,2,1,3).contiguous()

x_rsh1

#
#[batch
#[head1[head2 [head3 [head4
#점맛00 점맛00 점맛00 점맛00 
#[1xxx],[xxxx],[xxxx],[xxxx]      점 
#[12xx],[xxxx],[xxxx],[xxxx]      맛
#[123x],[xxxx],[xxxx],[xxxx]      0
#[1234],[xxxx],[xxxx],[xxxx]      0  유사도.
#      ]      ]      ]      ]]]
#점심, 점심 유사도
#점심, 맛있다 유사도, 맛있다, 맛있다 유사도  

#value_split
#[batch
#[head1[head2 [head3 [head4
#[xx],[xx],[xx],[xx]    점  -> 다 합치면 feature=10 길이 점심 벡터 됨
#[xx],[xx],[xx],[xx]    맛
#[xx],[xx],[xx],[xx]     0
#[xx],[xx],[xx],[xx]     0     각각의 위치에 대응하는 유사도 곱해짐.
#      ]      ]      ]      ]]]
#
#


tensor([[[[ 0.2355, -0.0143],
          [ 0.0716,  0.0137],
          [ 0.2355, -1.0561],
          [ 0.6391,  1.8472],
          [ 0.8236,  0.9426]],

         [[-1.9814,  0.2132],
          [ 0.4332, -0.4094],
          [ 0.8367, -0.4002],
          [ 1.2072,  0.5421],
          [ 1.1521, -0.3968]],

         [[-0.5885, -0.5346],
          [ 0.4459,  0.2157],
          [ 0.1549, -1.0978],
          [ 0.8451,  0.7277],
          [ 0.9705, -0.1014]],

         [[-0.5426, -0.2635],
          [ 0.3299,  0.0588],
          [ 0.2878, -0.6463],
          [ 0.6328,  0.5007],
          [ 0.7136, -0.0358]],

         [[-0.4341, -0.2108],
          [ 0.2639,  0.0470],
          [ 0.2303, -0.5170],
          [ 0.5062,  0.4006],
          [ 0.5709, -0.0287]]]], grad_fn=<CloneBackward0>)

In [ ]:
# 4. 헤드 concatenate하기
# (batch_size, query의 문장 길이, d_model)
x_concat = x_rsh1.view(batch_size,-1,d_model)

#concat
#[batch
#[headconcat = 1, 없음. 
#  점     맛     0       0 
#[x1x2],[xxxx],[xxxx],[xxxx] 점
#[x3x4],[xxxx],[xxxx],[xxxx] 맛  
#[x1x2],[xxxx],[xxxx],[xxxx] 0   
#[x3x4],[xxxx],[xxxx],[xxxx] 0      각각의 위치에 대응하는 유사도 곱해진 상태.
#      ]      ]      ]      ]]]

# 5. WO에 해당하는 밀집층 지나기
attn_output = out_proj(x_concat)

print(attn_output)
print(attn_output.shape)

#점심 output
#점심 맛있다 output
#점심 맛있다 0 output
#점심 맛있다 0 0 output

tensor([[[ 0.3817, -0.7419, -0.5343, -0.8161,  0.4207,  0.8646,  0.1605,
          -1.2684, -0.5796, -0.3462],
         [-0.1280, -0.5485, -0.1960, -1.0721,  0.1197,  0.1356,  0.7457,
          -0.4280, -0.1493,  0.7044],
         [-0.0635, -0.3292, -0.0773, -0.8207,  0.0515,  0.8330,  0.2200,
          -0.5880, -0.3021,  0.1980],
         [ 0.0360, -0.2048, -0.1765, -0.6820,  0.0103,  0.5177,  0.2018,
          -0.4236, -0.1613,  0.2135],
         [ 0.0851, -0.1063, -0.2017, -0.5961, -0.0502,  0.4262,  0.1230,
          -0.3455, -0.1249,  0.2003]]], grad_fn=<AddBackward0>)
torch.Size([1, 5, 10])


In [ ]:
attention1 = attn_output

In [ ]:
padding_mask = dec_expand_mask

# cross attention

In [ ]:
#attention2 = self.cross_attn(attention1, enc_hidden_states, padding_mask)
query, key, value, mask = attention1, enc_hidden_states, enc_hidden_states, dec_expand_mask

query

tensor([[[ 0.3817, -0.7419, -0.5343, -0.8161,  0.4207,  0.8646,  0.1605,
          -1.2684, -0.5796, -0.3462],
         [-0.1280, -0.5485, -0.1960, -1.0721,  0.1197,  0.1356,  0.7457,
          -0.4280, -0.1493,  0.7044],
         [-0.0635, -0.3292, -0.0773, -0.8207,  0.0515,  0.8330,  0.2200,
          -0.5880, -0.3021,  0.1980],
         [ 0.0360, -0.2048, -0.1765, -0.6820,  0.0103,  0.5177,  0.2018,
          -0.4236, -0.1613,  0.2135],
         [ 0.0851, -0.1063, -0.2017, -0.5961, -0.0502,  0.4262,  0.1230,
          -0.3455, -0.1249,  0.2003]]], grad_fn=<AddBackward0>)

In [ ]:
key.shape

torch.Size([1, 10, 10])

In [ ]:
batch_size = 1
num_heads = 5
d_model = 10 #feature
d_head = 2
#점심(x1x2), 먹는다(x1x2)
#점심(x3x4), 먹는다(x3x4).. 이렇게 나눠짐
query_split = query.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)
key_split = key.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)
value_split = value.view(batch_size, -1, num_heads, d_head).permute(0, 2, 1, 3)

print(query_split)
print(query_split.shape)

tensor([[[[ 0.3817, -0.7419],
          [-0.1280, -0.5485],
          [-0.0635, -0.3292],
          [ 0.0360, -0.2048],
          [ 0.0851, -0.1063]],

         [[-0.5343, -0.8161],
          [-0.1960, -1.0721],
          [-0.0773, -0.8207],
          [-0.1765, -0.6820],
          [-0.2017, -0.5961]],

         [[ 0.4207,  0.8646],
          [ 0.1197,  0.1356],
          [ 0.0515,  0.8330],
          [ 0.0103,  0.5177],
          [-0.0502,  0.4262]],

         [[ 0.1605, -1.2684],
          [ 0.7457, -0.4280],
          [ 0.2200, -0.5880],
          [ 0.2018, -0.4236],
          [ 0.1230, -0.3455]],

         [[-0.5796, -0.3462],
          [-0.1493,  0.7044],
          [-0.3021,  0.1980],
          [-0.1613,  0.2135],
          [-0.1249,  0.2003]]]], grad_fn=<PermuteBackward0>)
torch.Size([1, 5, 5, 2])


In [ ]:
d_K = torch.tensor(key.size()[-1])
scores = torch.matmul(query_split, key_split.permute(0, 1, 3, 2)) / torch.sqrt(d_K)

scores.shape

#attention: [n_batch, n_head, n_Q, n_K]

#scores는 
#[
#[head1[head2 [head3 [head4 [head5
#오점맛또 오점맛또 오점맛또 오점맛또
#[xxxx],[xxxx],[xxxx],[xxxx]      점
#[xxxx],[xxxx],[xxxx],[xxxx]      맛
#[xxxx],[xxxx],[xxxx],[xxxx]      0
#[xxxx],[xxxx],[xxxx],[xxxx]      0  유사도.
#      ]      ]      ]      ]]]

torch.Size([1, 5, 5, 10])

In [ ]:
scores.shape

torch.Size([1, 5, 5, 10])

In [ ]:
mask.shape

torch.Size([1, 1, 5, 10])

In [ ]:
if mask is not None:
    scores = scores.masked_fill(mask!=0,-1e9)
attention = torch.softmax(scores, dim=-1)

print(attention)
print(attention.shape)

tensor([[[[0.1759, 0.1725, 0.1647, 0.1707, 0.1459, 0.1702, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1701, 0.1747, 0.1589, 0.1706, 0.1561, 0.1696, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1689, 0.1714, 0.1622, 0.1690, 0.1601, 0.1685, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1687, 0.1689, 0.1651, 0.1680, 0.1616, 0.1678, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1682, 0.1673, 0.1669, 0.1672, 0.1632, 0.1672, 0.0000, 0.0000,
           0.0000, 0.0000]],

         [[0.1736, 0.1740, 0.1655, 0.1682, 0.1516, 0.1670, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1712, 0.1772, 0.1669, 0.1713, 0.1452, 0.1682, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1696, 0.1749, 0.1671, 0.1706, 0.1497, 0.1681, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1701, 0.1733, 0.1667, 0.1694, 0.1529, 0.1676, 0.0000, 0.0000,
           0.0000, 0.0000],
          [0.1701, 0.1724, 0.1665, 0.1688, 0.1548, 0.1674, 0.0000, 0.00

In [ ]:
x_raw = torch.matmul(attention,value_split)
x_rsh1 = x_raw.permute(0,2,1,3).contiguous()

# 4. 헤드 concatenate하기
# (batch_size, query의 문장 길이, d_model)
x_concat = x_rsh1.view(batch_size,-1,d_model)


# 5. WO에 해당하는 밀집층 지나기
attn_output = out_proj(x_concat)

print(attn_output)
print(attn_output.shape)


tensor([[[ 0.4636,  0.1492, -0.5238, -0.0665, -0.1144, -0.1222, -0.2308,
          -0.4546, -0.1806,  0.1621],
         [ 0.4633,  0.1411, -0.5271, -0.0708, -0.1094, -0.1168, -0.2261,
          -0.4672, -0.1879,  0.1624],
         [ 0.4623,  0.1455, -0.5280, -0.0703, -0.1142, -0.1222, -0.2274,
          -0.4600, -0.1845,  0.1636],
         [ 0.4637,  0.1439, -0.5286, -0.0711, -0.1144, -0.1202, -0.2277,
          -0.4628, -0.1861,  0.1632],
         [ 0.4643,  0.1436, -0.5291, -0.0715, -0.1151, -0.1198, -0.2280,
          -0.4636, -0.1868,  0.1632]]], grad_fn=<AddBackward0>)
torch.Size([1, 5, 10])


In [ ]:
attention2 = attn_output

In [ ]:
dec_hidden_states = attention2

# transformerConditionalGen forward

In [ ]:
lm_head = nn.Linear(10, 50265, bias = False)

In [ ]:
out =  lm_head(dec_hidden_states)
print(out)
print(out.shape)

tensor([[[ 0.1999,  0.1153, -0.2351,  ...,  0.1132,  0.1846, -0.2816],
         [ 0.1966,  0.1164, -0.2347,  ...,  0.1137,  0.1852, -0.2818],
         [ 0.1987,  0.1170, -0.2361,  ...,  0.1123,  0.1845, -0.2831],
         [ 0.1985,  0.1173, -0.2365,  ...,  0.1129,  0.1843, -0.2832],
         [ 0.1986,  0.1175, -0.2369,  ...,  0.1132,  0.1842, -0.2835]]],
       grad_fn=<UnsafeViewBackward0>)
torch.Size([1, 5, 50265])


In [ ]:
output = out.contiguous().view([1,-1, 50265]).squeeze()
print(output)
print(output.shape)

tensor([[ 0.1999,  0.1153, -0.2351,  ...,  0.1132,  0.1846, -0.2816],
        [ 0.1966,  0.1164, -0.2347,  ...,  0.1137,  0.1852, -0.2818],
        [ 0.1987,  0.1170, -0.2361,  ...,  0.1123,  0.1845, -0.2831],
        [ 0.1985,  0.1173, -0.2365,  ...,  0.1129,  0.1843, -0.2832],
        [ 0.1986,  0.1175, -0.2369,  ...,  0.1132,  0.1842, -0.2835]],
       grad_fn=<SqueezeBackward0>)
torch.Size([5, 50265])


In [ ]:
label_ids = label_ids.contiguous().view([1,-1]).squeeze()
print(label_ids)
print(label_ids.shape)

tensor([  482, 11482,     2,  -100,  -100])
torch.Size([5])


In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
lm_loss = criterion(output, label_ids)
print(lm_loss)

tensor(10.9601, grad_fn=<NllLossBackward0>)


# transformerConditionalGen generate

# 돌려보기

In [ ]:
#ROUGE-1 31.0 이상, ROUGE-2 20.0 이상, ROUGE-L 30.0 이상

In [ ]:
#num_epochs 20                    -50까지 가도 될듯
#train_batch_size 16              -256까지 시도
#lr 5e-5
#d_model 512                      -256 시도
#num_heads 8
#num_encoder_layers 3
#num_decoder_layers 3

#Rouge1 Rouge2 RougeL
#Test Rouge1 : 15.5131 | Test Rouge2 : 4.2002 | Test RougeL : 13.8485  -loss는 계속 수렴

In [ ]:
#num_epochs 50                   -50까지 가도 될듯
#train_batch_size 32              -256까지 시도
#lr 5e-5
#d_model 512                      -256 시도 (일단 성능 좋지 않음)
#num_heads 8
#num_encoder_layers 3
#num_decoder_layers 3

#Rouge1 Rouge2 RougeL
#Test Rouge1 : 33.1486 | Test Rouge2 : 23.3712 | Test RougeL : 31.7886     |

In [ ]:
#num_epochs 50                   -50까지 가도 될듯
#train_batch_size 32              -256까지 시도
#lr 1e-4                          -너무 수렴을 못하는것같아서 줄일까? 근데 안나와서 늘렸음 
#d_model 512                      
#num_heads 8
#num_encoder_layers 3
#num_decoder_layers 3

#Rouge1 Rouge2 RougeL

In [142]:
!python main.py

Train Epoch: 100% 188/188 [00:32<00:00,  5.74it/s]
|                         Epoch / Total Epoch : 0 / 50                          |
|                              Train Loss : 7.7179                              |
Validation Loss: 100% 10/10 [00:00<00:00, 18.54it/s]
|                              Valid Loss : 6.8735                              |
Validation Rouge: 100% 300/300 [00:20<00:00, 14.42it/s]
<s>the is a is a to be to the.</s>
<s>the is a is a is a to the.</s>
<s>the is a is a is a to be.</s>
<s>the is a is a is a to be.</s>
<s>the is a is a to be to be.</s>
<s>the is a is a the to the the.</s>
<s>the is a is a to be to the.</s>
<s>the is a is a is a to be.</s>
<s>the is a is a to be to the the.</s>
<s>the is a is a the the the the the.</s>
|     Valid Rouge1 : 9.2545 | Valid Rouge2 : 0.3985 | Valid RougeL : 8.7289     |
Train Epoch: 100% 188/188 [00:33<00:00,  5.66it/s]
|                         Epoch / Total Epoch : 1 / 50                          |
|                       

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
id = torch.tensor([47471])

In [ ]:
tokenizer.decode(id.view([-1]))

'ileaks'